In [1]:
from transformers import GPT2TokenizerFast, GPT2Tokenizer
from copy import deepcopy
import numpy as np
from random import random
from time import time

/home/maciej/anaconda3/envs/tokenizex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from data_loader import get_imput
from embedding_mask import create_mask_embedding, sum_masks, process_token, encode_em


In [3]:
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
vocab = tokenizer.get_vocab()
vocabi = {v:k for k, v in vocab.items()}

/home/maciej/anaconda3/envs/tokenizex/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [34]:

def tokenize_atoms(txt, tokenizer):
    atxt = "".join(tokenizer.tokenize(txt))
    atxt = list(atxt)
    eatxt = tokenizer.encode(atxt)
    return eatxt

c = 0

def lazy_past_tokens_gather(atokens, tokenizer): #dev no dobra, może tak być dla tokenów kłopotliwych, ale dla reszty już nie powinno
    global c #dev
    c+=1 #dev
    past_emb_pos = []
    for i in range(1, len(atokens)+1):
        past_emb_pos.append(atokens[:i])
    return past_emb_pos

def past_tokens_gather(atokens, tokenizer):
    past_emb_pos = []
    for i in range(1, len(atokens)+1):
        pdec = tokenizer.decode(atokens[:i])
        if '�' in pdec:
            return lazy_past_tokens_gather(atokens, tokenizer)
        penc = tokenizer.encode(pdec)
        past_emb_pos.append(penc)
    return past_emb_pos


def create_emb_pos_mask(emb_pos):
    res = []
    # mask = np.eye(len(emb_pos)).astype(bool)
    mask = np.eye(len(emb_pos))
    for embt, m in zip(emb_pos, mask):
        res.append(embt[-1])
        for ie in embt[:-1]:
            is_added = True if res[:-1] == [] else False
            for j, je in enumerate(res[:-1]):
                if ie == je:
                    # m[j] = True
                    m[j] = 1
                    is_added = True
                    break
            if not is_added:
                raise Exception("target len != input len")
    return np.array(res)[:,0], np.array(res)[:,1], mask

def tokenizex_encode_full(txt, tokenizer):
    tenc = tokenize_atoms(txt, tokenizer)
    res = past_tokens_gather(tenc, tokenizer)
    g = [list(zip(e, np.arange(len(e)))) for e in res]
    e, p, m = create_emb_pos_mask(g)
    return e, p, m


In [35]:
# def process_token(t, tokenizer):
#     td = tokenizer.decode(t)
#     if len(td)==1 or "�" in td:
#         emb = np.array([t]) 
#         pos = np.zeros((1))
#         mask = np.ones((1,1))
#     else:
#         emb, pos, mask = tokenizex_encode_full(td, tokenizer)
#         if len(emb) == 1:
#             raise Exception("Used!")
        
    # return emb, pos, mask

def create_atomizer(tokenizer):
    """
    {
        "token_id":{
            "embedding":np.array(int), 
            "pe":np.array(int), 
            "mask":np.array(np.array(union(0, 1)))
        }
    }

    """
    tokenizex_atomizer = {}
    vocab = tokenizer.get_vocab()
    vocabi = {v:k for k, v in vocab.items()}

    for tok, txt in vocabi.items():
        e, p, m = tokenizex_encode_full(tokenizer.decode(tok), tokenizer)
        tokenizex_atomizer[tok] = (e, p, m)
        
    return tokenizex_atomizer
        
def tokenizex_encode(txt, tokenizer, atomizer):
    tokens = tokenizer.encode(txt)

    res_emb = []
    res_pos = []
    partial_masks = []
    pos_len = 0
    
    for t in tokens:
        emb, pos, mask = atomizer[t]
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)   
    
    full_mask = sum_masks(len(res_emb), partial_masks)
    return np.array(res_emb), np.array(res_pos), full_mask


In [36]:
c = 0 
atomizer = create_atomizer(tokenizer)
print(c)

873


In [37]:
def validate(txt, emb, pos, mask, tokenizer):
    dec = tokenizer.decode(emb[mask[-1].astype(bool)])
    assert dec == txt

    lemb = tokenize_atoms(txt, tokenizer)
    assert len(lemb) == len(emb)

    comp = pos[mask[-1].astype(bool)]
    for a, b in zip(comp.astype(int), list(range(len(comp)))):
        assert a == b

In [38]:
txt = "  ż"

ee, pp, mm = tokenizex_encode(txt, tokenizer, atomizer)

validate(txt, ee, pp, mm, tokenizer)

AssertionError: 

In [ ]:
def tokenizex_encode_dev(txt, tokenizer, atomizer):
    tokens = tokenizer.encode(txt)

    res_emb = []
    res_pos = []
    partial_masks = []
    pos_len = 0
    
    for t in tokens:
        emb, pos, mask = atomizer[t]
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)   
    
    full_mask = sum_masks(len(res_emb), partial_masks)
    return np.array(res_emb), np.array(res_pos), full_mask

In [41]:
# txt = """  𡨸 𡨸żóąs"""
txt = "  ż"

tenc = tokenize_atoms(txt, tokenizer)
res = past_tokens_gather(tenc, tokenizer)
g = [list(zip(e, np.arange(len(e)))) for e in res]
e, p, m = create_emb_pos_mask(g)

validate(txt, e, p, m, tokenizer)

In [44]:
tokenizer.encode(txt)

[220, 25370, 120]

In [47]:
atomizer[120]

(array([171, 123, 121]),
 array([0, 1, 2]),
 array([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]]))

In [42]:
e

array([220, 220, 129, 120])

In [ ]:
e, p, m = tokenizex_encode_full(txt, tokenizer)
tokenizer.decode(e[m[-1]])

'  𡨸 𡨸żóąs'

In [7]:
num = 12331
print(atomizer[num])
d = tokenizer.decode(num)
print(d)
tokenize_atoms(d, tokenizer)

(array([   36,  9139,    81,   305, 12331]), array([0, 0, 1, 1, 0]), array([[ True, False, False, False, False],
       [False,  True, False, False, False],
       [False,  True,  True, False, False],
       [False,  True, False,  True, False],
       [False, False, False, False,  True]]))
Error


[36, 81, 81, 78, 81]

In [ ]:
at = tokenize_atoms(txt, tokenizer)
at

[220,
 220,
 172,
 94,
 101,
 116,
 220,
 172,
 94,
 101,
 116,
 129,
 120,
 127,
 111,
 128,
 227,
 82]

In [ ]:
past_tokens_gather(at, tokenizer)

[[220],
 [220, 220],
 [220, 220, 172],
 [220, 220, 172, 94],
 [220, 220, 172, 94, 101],
 [220, 220, 172, 94, 101, 116],
 [220, 220, 172, 94, 101, 116, 220],
 [220, 220, 172, 94, 101, 116, 220, 172],
 [220, 220, 172, 94, 101, 116, 220, 172, 94],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101, 116],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101, 116, 129],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101, 116, 129, 120],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101, 116, 129, 120, 127],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101, 116, 129, 120, 127, 111],
 [220,
  220,
  172,
  94,
  101,
  116,
  220,
  172,
  94,
  101,
  116,
  129,
  120,
  127,
  111,
  128],
 [220,
  220,
  172,
  94,
  101,
  116,
  220,
  172,
  94,
  101,
  116,
  129,
  120,
  127,
  111,
  128,
  227],
 [220,
  220,
  172,
  94,
  101,
  116,
  220,
  172,
  94,
  101,
  116,
  129,
  120,
  127,
  111,
  128,
  227,
  82]]

In [ ]:
tokenizer.tokenize(" ó")

['ĠÃ', '³']

In [ ]:
tokenizer.decode([220, 127, 111])

' ó'

In [ ]:
res

[[220],
 [220, 220],
 [220, 220, 172],
 [220, 220, 172, 94],
 [220, 220, 172, 94, 101],
 [220, 220, 172, 94, 101, 116],
 [220, 220, 172, 94, 101, 116, 220],
 [220, 220, 172, 94, 101, 116, 220, 172],
 [220, 220, 172, 94, 101, 116, 220, 172, 94],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101, 116],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101, 116, 129],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101, 116, 129, 120],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101, 116, 129, 120, 127],
 [220, 220, 172, 94, 101, 116, 220, 172, 94, 101, 116, 129, 120, 127, 111],
 [220,
  220,
  172,
  94,
  101,
  116,
  220,
  172,
  94,
  101,
  116,
  129,
  120,
  127,
  111,
  128],
 [220,
  220,
  172,
  94,
  101,
  116,
  220,
  172,
  94,
  101,
  116,
  129,
  120,
  127,
  111,
  128,
  227],
 [220,
  220,
  172,
  94,
  101,
  116,
  220,
  172,
  94,
  101,
  116,
  129,
  120,
  127,
  111,
  128,
  227,
  82]]

In [ ]:
l = []
for r in m:
    l.append(e[r.astype(bool)].tolist())
l == res


True

In [ ]:

def time_valid_tokenization(txt_data, tokenizer, def_tokenizer, C, lbl_val = False):
    st = 0
    for i in range(C):
        txti = def_tokenizer.decode(def_tokenizer.encode(get_imput(txt_data, 512)))
        t1 = time()
        emb, pos, mask = tokenizer(txti)
        st += time()-t1

        if lbl_val: 
            lemb = tokenize_atoms(txti, def_tokenizer)
            if len(lemb) != len(emb):
                print(txti)
            assert len(lemb) == len(emb)

            comp = pos[mask[-1].astype(bool)]
            for a, b in zip(comp.astype(int), list(range(len(comp)))):
                assert a == b

    return st


In [ ]:
txt = """ ósiem"""
emb, pos, mask = encode_em(txt, tokenizer)
validate(txt, emb, pos, mask, tokenizer)

AssertionError: 

In [ ]:
txt = " ósiem"

tokens = tokenizer.tokenize(txt)
res_emb = []
res_pos = []
partial_masks = []
pos_len = 0

for t in tokens:
    emb, pos, mask = process_token(t, tokenizer)

    res_emb.extend(emb)
    res_pos.extend(pos+pos_len)
    pos_len += pos[-1] + 1
    partial_masks.append(mask)



In [ ]:
t1  = "elo źle żal ósemka no i śląska"
t3 = "example text to tokenize and, żargon ołówka źle wpływa detokenize, ślóza."
t1  = "elo źle żaló"
t3 = "example."
e2  = np.array([69])
p2  = np.array([0])
m2  = np.array([1])
t2 = tokenizer.decode(e2)

e1, p1, m1 = encode_em(t1, tokenizer)
e3, p3, m3 = encode_em(t3, tokenizer)
txt = t1+t2+t3
txt

emb = np.concatenate([e1,e2,e3])
pos = sum_pos([p1,p2,p3])
mlen = m1.shape[0]+m2.shape[0]+m3.shape[0]
mask = sum_masks(mlen, [m1, m2, m3])


In [ ]:
# faulty examples
' ósiem'
"ó"
""" ›"""
''

In [ ]:

def text_emb_pos(txt, tokenizer):
    emb_pos = []

    letter_split = False
    for l in txt:
        t = tokenizer.encode(l)
        if len(t) > 1:
            letter_split = True
    
    if letter_split:
        for l in txt:
            et = tokenizer.encode(l)
            emb_pos.append(list(zip(et, list(range(len(et))))))
    else:
        for i in range(1, len(txt)+1):
            et = tokenizer.encode(txt[:i])
            emb_pos.append(list(zip(et, list(range(len(et))))))

    return emb_pos

In [ ]:
for t in atomizer.keys():
    txt = tokenizer.decode(t)
    emb = tokenize_letters(txt, tokenizer)
    a_emb = atomizer[t][0]
    assert len(emb) == len(a_emb)
    if len(emb) != len(a_emb):
        print(len(emb), len(a_emb))

In [ ]:
def tokenize_letters(txt, tokenizer):
    emb = []
    for l in txt:
        emb.extend(tokenizer.encode(l))
    return emb
